Importação da biblioteca `google.colab.drive` e montagem do Google Drive usando o método `drive.mount('/content/drive')`, no ambiente do Colab para acessar os arquivos e diretórios no drive. É necessário autorizar o acesso ao drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Instalação das bibliotecas `transformers` e `datasets` usando o comando `!pip install`. As bibliotecas `transformers` fornecem muitos modelos pré-treinados e utilidades para processamento de linguagem natural, enquanto a biblioteca `datasets` contém vários conjuntos de dados para treinamento e avaliação de modelos.

In [ ]:
!pip install transformers
!pip install datasets

Definição da versão do pacote `pytorch-xla` e instalação usando o comando `!curl` para baixar o script de configuração e `!python` para executar o script. Isso é necessário para executar o código em um ambiente distribuído usando o TPU (Unidade de Processamento Tensorial).

In [ ]:
VERSION = "1.8.1"
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

Definição de variáveis, como `squad_v2` (que indica se o conjunto de dados SQuAD é a versão 2), `model_checkpoint` (que especifica o modelo BERT pré-treinado em português a ser usado) e `batch_size` (o tamanho do lote de dados usado durante o treinamento).

In [ ]:
squad_v2 = False
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
batch_size = 4

Importação das funções `load_dataset` e `load_metric` da biblioteca `datasets`. Essas funções são usadas para carregar conjuntos de dados e métricas de avaliação para treinar os modelos.

In [ ]:
from datasets import load_dataset, load_metric

Download do conjunto de dados SQuAD em português usando o comando `!wget`. O comando baixa um arquivo .tar.gz do Google Drive e salva-o como squad-pt.tar.gz. A opção --load-cookies é usada para especificar o local de um arquivo de cookies temporário usado para autenticar o download.

In [ ]:
# Get dataset SQUAD in Portuguese
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn" -O squad-pt.tar.gz && rm -rf /tmp/cookies.txt


Extrai o arquivo .tar.gz usando o comando `!tar -xvf squad-pt.tar.gz`. O comando extrai os arquivos do conjunto de dados SQuAD em português.

In [ ]:
!tar -xvf squad-pt.tar.gz

A célula contém um bloco de código que é executado com o temporizador ativado (`%%time`). Ele é responsável por ler os arquivos JSON do SQuAD em português e convertê-los em um novo formato exigido pelo modelo de perguntas e respostas. A célula itera sobre os arquivos JSON, extrai informações relevantes, como contexto, pergunta e respostas, e as armazena em um novo dicionário. Esse novo arquivo é salvo no disco com um novo nome (prefixado com 'pt_').

In [ ]:
%%time
# new

# Get the train and validation json file in the HF script format
# inspiration: file squad.py at https://github.com/huggingface/datasets/tree/master/datasets/squad

import json

files = ['squad-train-v1.1.json','squad-dev-v1.1.json']

for file in files:

    # Opening JSON file & returns JSON object as a dictionary
    f = open(file, encoding="utf-8")
    data = json.load(f)

    # Iterating through the json list
    entry_list = list()
    id_list = list()

    for row in data['data']:
        title = row['title']

        for paragraph in row['paragraphs']:
            context = paragraph['context']

            for qa in paragraph['qas']:
                entry = {}

                qa_id = qa['id']
                question = qa['question']
                answers = qa['answers']

                entry['id'] = qa_id
                entry['title'] = title.strip()
                entry['context'] = context.strip()
                entry['question'] = question.strip()

                answer_starts = [answer["answer_start"] for answer in answers]
                answer_texts = [answer["text"].strip() for answer in answers]
                entry['answers'] = {}
                entry['answers']['answer_start'] = answer_starts
                entry['answers']['text'] = answer_texts

                entry_list.append(entry)

    reverse_entry_list = entry_list[::-1]

    # for entries with same id, keep only last one (corrected texts by he group Deep Learning Brasil)
    unique_ids_list = list()
    unique_entry_list = list()
    for entry in reverse_entry_list:
        qa_id = entry['id']
        if qa_id not in unique_ids_list:
            unique_ids_list.append(qa_id)
            unique_entry_list.append(entry)

    # Closing file
    f.close()

    new_dict = {}
    new_dict['data'] = unique_entry_list

    file_name = 'pt_' + str(file)
    with open(file_name, 'w') as json_file:
        json.dump(new_dict, json_file)

Carrega os conjuntos de dados de treinamento e validação usando `load_dataset` da biblioteca `datasets`. Os arquivos JSON recém-processados são especificados e os nomes dos campos são definidos para que o pipeline de treinamento as reconheça corretamente.

In [ ]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json',
                        data_files={'train': 'pt_squad-train-v1.1.json', 'validation': 'pt_squad-dev-v1.1.json'},
                        field='data')

Define uma função chamada `show_random_elements` que exibe aleatoriamente algumas entradas do conjunto de dados. A função usa a biblioteca `pandas` para exibir os dados em formato de tabela.

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

Mostra exemplos aleatórios do conjunto de treinamento usando a função `show_random_elements`.

In [ ]:
show_random_elements(datasets["train"])

Importação do módulo `AutoTokenizer` da biblioteca `transformers` e criação de um objeto `tokenizer` que será usado para tokenizar os dados de entrada antes de alimentá-los para o modelo. O método `from_pretrained` é usado para carregar o tokenizador pré-treinado correspondente ao modelo BERT especificado.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Definição das variáveis `max_length` e `doc_stride`. `max_length` é o comprimento máximo de uma entrada (pergunta e contexto) que será alimentada para o modelo. `doc_stride` é a sobreposição autorizada entre duas partes do contexto ao dividi-lo.

Itera sobre o conjunto de dados de treinamento para encontrar o primeiro exemplo cujo comprimento de entrada (pergunta + contexto) excede `max_length`. O exemplo é armazenado na variável `example` para fins de depuração. Em seguida, o comprimento da entrada tokenizada é impresso.

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
        break
example = datasets["train"][i]

print(example)

len(tokenizer(example["question"], example["context"])["input_ids"])

Verifica se o tokenizador insere o preenchimento à direita (`right`). Essa informação será usada posteriormente para criar recursos de treinamento.

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

Define a função `prepare_train_features` que é usada para preparar os exemplos de treinamento. Os exemplos são tokenizados, divididos em vários recursos (se necessário) e as respostas são mapeadas para as posições tokenizadas correspondentes. O resultado é um dicionário com os tokens de entrada e as posições das respostas.

In [ ]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

Aplica a função `prepare_train_features` aos conjuntos de dados de treinamento e validação mapeados. O resultado é armazenado na variável `tokenized_datasets`.

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Define o nome de arquivo com base no modelo BERT usado (`model_checkpoint`) e salva o modelo treinado e o tokenizador no diretório `model_trained`.

In [ ]:
file_name = str(model_checkpoint).replace('/','-') + '-ric_finetuned-squad-v1.1-pt'

Importa as bibliotecas `transformers`, `AutoModelForQuestionAnswering`, `TrainingArguments` e `Trainer`. Cria uma instância do modelo BERT pré-treinado usando o método `from_pretrained`. Em seguida, envolve o modelo em um objeto `MpModelWrapper` da biblioteca `torch_xla` para treinamento distribuído usando o TPU.

In [ ]:
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch_xla.distributed.xla_multiprocessing as xmp

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

WRAPPED_MODEL = xmp.MpModelWrapper(model)

Importação da função `default_data_collator` da biblioteca `transformers`. Criação de um objeto `data_collator` que será usado para agrupar os dados de treinamento durante o treinamento do modelo.

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

Define a função de treinamento `training` que é chamada durante a fase de treinamento com `Trainer`. A função define os argumentos de treinamento, cria uma instância de `Trainer` e treina o modelo. O modelo treinado e o tokenizador são salvos no diretório `model_trained`.

In [ ]:
def training(model):
    args = TrainingArguments(
        f"{file_name}",
        num_train_epochs=4.0,
        evaluation_strategy = "epoch",
        save_strategy = "epoch",
        weight_decay=0.001,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        #save_total_limit=1
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
    )
    trainer.place_model_on_device = False
    trainer.train()
    trainer.save_model("model_trained/")
    tokenizer.save_pretrained("model_trained/")

Criação de código distribuído usando a biblioteca `torch_xla`. A função `_mp_fn` é usada para criar uma instância de modelo em cada processo distribuído usando uma determinada estratégia (`xm.xla_device`). O modelo é treinado chamando a função `training`.

Função `main` que chama a função `_mp_fn` para iniciar o treinamento distribuído. O número de processos a serem executados pode ser especificado usando o argumento `nprocs`.

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
import warnings

warnings.filterwarnings("ignore")

def _mp_fn(index):
    device = xm.xla_device()

    model = WRAPPED_MODEL.to(device)

    training(model)

xmp.spawn(_mp_fn, nprocs=8, start_method="fork")

Código para mover do treinamento local para o Google Drive usando o módulo `pathlib` para manuseio de caminhos de arquivos. O diretório de trabalho atual (`dest/`) é movido para o diretório `model_trained` no Google Drive.

In [ ]:
import pathlib
from pathlib import Path

dest = pathlib.Path.cwd()
fname_HF = 'model_trained'

path_to_model = dest/fname_HF

Importação do módulo `Pipeline` da biblioteca `transformers`. Criação de um objeto `nlp` para responder a perguntas usando o modelo treinado. O modelo treinado é especificado no argumento `model`.

In [ ]:
from transformers import pipeline

model_qa = str(path_to_model)
nlp = pipeline("question-answering", model=model_qa)

Definição de um contexto no qual uma pergunta será feita. O contexto é uma passagem de texto em que a pergunta será respondida.

In [ ]:
context = r"""
Integralização de um curso é quando o estudante cumpre todas as exigências fixadas no Catálogos de cursos.
O sistema de integralização adotado pela Universidade é o de crédito e matrícula por disciplina.
O relatório de Integralização Curricular é publicado na web semestralmente, constando a relação de disciplinas cursadas com suas respectivas informações - se é obrigatória, eletiva ou extracurricular, créditos e nota obtida -, as disciplinas em que o estudante está matriculado atualmente, as disciplinas que ainda precisam ser cursadas e o prazo máximo para conclusão do curso, dentre outras informações.
O estudante pode consultar esse documento no site da DAC, em Integralização Curricular, após logar em Serviços Acadêmicos.
Prazo mínimo e máximo são os tempos mínimo e máximo que o estudante dispõe para a conclusão do curso. O tempo mínimo é igual ao número de períodos fixados pelas Unidades de Ensino na proposta para o cumprimento do Currículo Pleno. O tempo máximo corresponde à soma do tempo mínimo e mais 50% desse período (exemplo: para um curso de 08 semestres de período mínimo, o prazo máximo seria de 12 semestres).
Do tempo máximo de integralização deve ser subtraído o tempo referente a aproveitamento de estudos, dispensas de disciplinas e teste de proficiência. Isso quer dizer que o período máximo pode ser menor do que o estipulado inicialmente quando o estudante solicitar aproveitamento e/ou teste de proficiência.
Caso o estudante solicite trancamento de semestre, o prazo máximo de integralização será acrescido do mesmo número de períodos letivos em que a matrícula esteve trancada. Isso quer dizer que o período máximo será maior que o estipulado inicialmente quando o estudante solicitar trancamento de semestre.
O estudante pode verificar seu prazo de integralização no site da DAC, em Integralização Curricular, após logar em Serviços Acadêmicos.
"""

Definição de uma pergunta que será feita sobre o contexto. A pergunta é especificada no argumento `question` ao chamar o objeto `nlp` para obter a resposta.

Impressão da resposta obtida pelo modelo, a pontuação atribuída à resposta e as posições de início e fim da resposta no contexto original.

In [ ]:
question = "O que tem no relatório de integralização?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Definição de um contexto no qual uma pergunta será feita. O contexto é uma passagem de texto em que a pergunta será respondida.

In [ ]:
context = r"""
Certificado de Estudos é o documento que atesta a conclusão de um elenco de disciplinas integrantes de um ramo específico do conhecimento.
O oferecimento de Certificados de Estudos bem como a definição do elenco de disciplinas que os compõem ficam a critério da Coordenadoria de Curso responsável pelas disciplinas envolvidas.
Os Certificados de Estudos são emitidos pela Diretoria Acadêmica (DAC) quando solicitados pelos estudantes regulares ou especiais que forem aprovados em todas as disciplinas exigidas.
Mais informações podem ser obtidas no Regimento Geral dos cursos de Graduação.
Procedimentos: Para solicitar o Certificado de Estudos, o estudante deve protocolar seu pedido pelo sistema e-DAC, utilizando o assunto 'Certificado de Estudos' e citando, no campo de texto, o nome do certificado, o catalogo do qual o certificado faz parte(curso/ano) e as disciplinas foram cursadas para este fim.
Caso seja necessário, a Diretoria Acadêmica(DAC) irá solicitar que a Coordenadoria de Graduação responsável emita um ofício, validando o cumprimento das exigidas. Constatando o cumprimento das exigências, a DAC emitirá o Certificado e avisará quando o mesmo estiver disponível para a retirada.
"""

Definição de uma pergunta que será feita sobre o contexto. A pergunta é especificada no argumento `question` ao chamar o objeto `nlp` para obter a resposta.

Impressão da resposta obtida pelo modelo, a pontuação atribuída à resposta e as posições de início e fim da resposta no contexto original.

In [ ]:
question = "Como faço para pedir um certificado de estudos?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Definição de um contexto no qual uma pergunta será feita. O contexto é uma passagem de texto em que a pergunta será respondida.

In [ ]:
context = r"""
Para efeito de cálculo de carga horária, serão atribuídas a cada disciplina unidades de crédito, sendo que cada unidade de crédito corresponde a 15 (quinze) horas/aula, com duração de 15 (quinze) semanas, para as disciplinas semestrais.
O total de créditos a ser cumprido pelo estudante para conclusão do curso será estabelecido pelo Catálogo dos Cursos de Graduação.
Pré-requisitos são condições consideradas indispensáveis para matrícula em disciplinas, podendo ser classificado como:
Pré-requisito pleno: disciplina ou o conjunto de disciplinas em que o estudante deve obter aprovação para matricular-se em outra disciplina.
Pré-requisito parcial: a disciplina ou o conjunto de disciplinas em que o estudante deve obter frequência mínima e média final maior ou igual a 3,0 (três) para matricular-se em outra disciplina.
Pré-requisito especial: condição não atrelada a disciplinas específicas, podendo ser AA4nn - (pela exigência de Coeficiente de Progressão maior ou igual a 0,xx) ou AA200 (por autorização da Coordenadoria que a oferece).
Não é autorizada a utilização da AA200 para disciplinas obrigatórias, devendo os casos excepcionais ser devidamente justificados e remetidos à CCG para apreciação Todos os pré-requisitos exigidos para as disciplinas do curso de graduação constam no Currículo Pleno seguido pelo estudante.
"""

Definição de uma pergunta que será feita sobre o contexto. A pergunta é especificada no argumento `question` ao chamar o objeto `nlp` para obter a resposta.

Impressão da resposta obtida pelo modelo, a pontuação atribuída à resposta e as posições de início e fim da resposta no contexto original.

In [ ]:
question = "O que é um pré requisito parcial?"

result = nlp(question=question, context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Cria uma nova pasta chamada `model_trained_v30` no Google Drive e copia o conteúdo do diretório `model_trained` para a nova pasta.

In [ ]:
!cp -r model_trained/ drive/MyDrive/model_trained_v30/